In [ ]:
# Built-in Imports
import sys
import os
from typing import Literal

sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("."))

# Third-Party Imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from prophet import Prophet


# Internal Imports
from utils.access_token import AccessToken
from utils._constants import (
    PAYLOAD,
    DOMAIN,
    API_VERSION,
)
from utils.utils import get_data

# Auth Setup
auth = AccessToken(domain=DOMAIN, payload=PAYLOAD)
auth.generate_access_token()
auth_header = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {auth.access_token}",
}

# Salesforce Queries
api_endpoint = DOMAIN + f"/services/data/v{API_VERSION}/query/?q="


# 1. Data Gathering

### 1.1. Get Forecast Data from Salesforce

In [ ]:
forecast_query = """SELECT 
Account__c, Account__r.Name, CreatedDate, Date__c, Amount__c, CreatedById, CreatedBy.Name, Account__r.Region__c, CurrencyIsoCode, Business_line__c, Product_Family__c 
FROM Forecast__c 
WHERE Account__c != null
"""
forecast_query = forecast_query.replace("\n", "").replace(" ", "+").strip()

# Get Forecast Data
raw_forecast_data = get_data(
    domain=DOMAIN,
    api_endpoint=api_endpoint,
    query=forecast_query,
    auth_header=auth_header,
    val1="Account__r",
    val2="CreatedBy",
)

In [ ]:
raw_forecast_data["CreatedDate"] = pd.to_datetime(
    raw_forecast_data["CreatedDate"], format="%Y-%m-%dT%H:%M:%S.000+0000"
)

raw_forecast_data["Date__c"] = pd.to_datetime(
    raw_forecast_data["Date__c"], format="%Y-%m-%d"
)

raw_forecast_data = raw_forecast_data.dropna(subset=['Product_Family__c'])

raw_forecast_data

### 1.2. Get Sales Data from Tableau

In [ ]:
def get_sales_data(sales_path: str) -> pd.DataFrame:
    sales_data = pd.read_csv(
        filepath_or_buffer=sales_path, date_format="%d-%b-%y", parse_dates=["Date"]
    )
    sales_data = sales_data.pivot(
        values="Measure Values",
        columns=["Measure Names"],
        index=[
            "Account Id",
            "Account Name",
            "Business Line",
            "Region",
            "Channel",
            "Product Family (Account Hierarchy)",
            "Product Subfamily (Account Hierarchy)",
            "Product Id",
            "Local Item Code (Zita)",
            "Local Item Description (Zita)",
            "Date",
        ],
    ).reset_index()
    sales_data = sales_data.rename(
        columns={
            "Product Family (Account Hierarchy)": "Product Family",
            "Product Subfamily (Account Hierarchy)": "Product Subfamily",
            "Local Item Code (Zita)": "Local Item Code",
            "Local Item Description (Zita)": "Local Item Description",
        }
    )

    sales_data["Business Line"] = (
        sales_data["Business Line"].replace(np.nan, "Unallocated (Unallocated)").copy()
    )
    sales_data["BL Short"] = (
        sales_data["Business Line"].str.findall(r"\((.*?)\)").str[0].copy()
    )

    return sales_data

sales_path = r"C:\Users\Lian.Zhen-Yang\BASF 3D Printing Solutions GmbH\Revenue Operations - General\06. Requests\SNOP FC Breakdown\SCM Forecast - Sales Topic_Summary.csv"
sales_data = get_sales_data(sales_path)
sales_data.head()

#### 1.2.1. Define model and plot function for prophet

In [ ]:
def model_and_plot(
    sales_data: pd.DataFrame,
    filters: dict = {},
    x: Literal["Year", "Month", "Date"] = "Date",
    y: Literal[
        "Total Sales", "Material Quantity (kg)", "Quantity in SKUs"
    ] = "Total Sales",
    dimension: str = "Dimension",
    resampling_period: str = "D",  # should be same as frequency
    future_period: int = 365,  # unit based on frequency
    frequency: str = "D",  # should be same as resampling_period
    prophet_kwargs: dict = {},
    sales_grouping_period: Literal["month", "quarter", "year"] = "month",
) -> pd.DataFrame:
    sales_data["Year"] = sales_data["Date"].dt.year
    sales_data["Month"] = sales_data["Date"].dt.month_name()
    sales_data["Month"] = pd.Categorical(
        sales_data["Month"],
        categories=[
            "January",
            "February",
            "March",
            "April",
            "May",
            "June",
            "July",
            "August",
            "September",
            "October",
            "November",
            "December",
        ],
        ordered=True,
    )

    # Remove negative sales values
    sales_data = sales_data[sales_data[y] >= 0]

    # filter out data based on kwargs
    if filters != {}:
        filtered_sales_data = sales_data.query(
            " & ".join([f"`{key}` == '{value}'" for key, value in filters.items()])
        )
        if filtered_sales_data.empty:
            raise ValueError("No data found for the given filters.")

    # create group columns list
    group_columns = [x] + list(filters.keys())
    agg_sales_data = filtered_sales_data.groupby(group_columns)[y].sum().reset_index()

    # Calculate additional summary metrics
    total_sales = filtered_sales_data[y].sum()
    average_sales = filtered_sales_data[y].mean()
    sales_std_dev = filtered_sales_data[y].std()
    number_of_days_ordered = filtered_sales_data["Date"].nunique()
    order_frequency = len(filtered_sales_data) / number_of_days_ordered
    median_sales = filtered_sales_data[y].median()
    total_transactions = len(filtered_sales_data)
    sales_variance = filtered_sales_data[y].var()
    average_order_value = total_sales / total_transactions
    sales_skewness = filtered_sales_data[y].skew()
    sales_kurtosis = filtered_sales_data[y].kurt()
    unique_customers = filtered_sales_data["Account Id"].nunique()
    repeat_customers = (
        filtered_sales_data.groupby("Account Id")
        .filter(lambda x: len(x) > 1)["Account Id"]
        .nunique()
    )
    repeat_customer_rate = (
        repeat_customers / unique_customers if unique_customers > 0 else 0
    )

    # Create a summary of sales data in filtered_sales_data
    summary_data = {
        "Metric": [
            "Total Sales",
            "Average Sales",
            "Sales Std Dev",
            "Number of Days Ordered",
            "Order Frequency",
            "Median Sales",
            "Total Transactions",
            "Sales Variance",
            "Average Order Value",
            "Sales Skewness",
            "Sales Kurtosis",
            "Unique Customers",
            "Repeat Customer Rate",
        ],
        "Value": [
            round(total_sales),
            round(average_sales),
            round(sales_std_dev),
            number_of_days_ordered,
            round(order_frequency),
            median_sales,
            total_transactions,
            round(sales_variance),
            round(average_order_value),
            round(sales_skewness),
            round(sales_kurtosis),
            unique_customers,
            round(repeat_customer_rate),
        ],
        "Unit": [
            "€",
            "€",
            "€",
            "days",
            "orders/day",
            "€",
            "transactions",
            "€",
            "€",
            "skewness",
            "kurtosis",
            "customers",
            "%",
        ],
    }
    filtered_sales_summary = pd.DataFrame(summary_data)

    # Create summary table for the plot
    filtered_sales_table = go.Table(
        header=dict(
            values=filtered_sales_summary.columns,
            fill_color="paleturquoise",
            align="left",
        ),
        cells=dict(
            values=[
                filtered_sales_summary[col] for col in filtered_sales_summary.columns
            ],
            fill_color="lavender",
            align="left",
        ),
    )

    if not agg_sales_data.empty:
        agg_sales_data["Dimension"] = (
            agg_sales_data[list(filters.keys())].astype(str).agg(" - ".join, axis=1)
        )

    resampled_df = (
        agg_sales_data.groupby(["Date"])["Total Sales"]
        .sum()
        .reset_index()
        .resample(resampling_period, on="Date")
        .sum()
        .reset_index()
    )

    resampled_df = resampled_df.rename(columns={"Date": "ds", "Total Sales": "y"})

    # fit linear regression model
    model_prophet = Prophet(**prophet_kwargs)

    model_prophet.fit(resampled_df)

    df_future = model_prophet.make_future_dataframe(
        periods=future_period, freq=frequency
    )

    forecast_prophet = model_prophet.predict(df_future)

    # combine test and forecast data
    forecast_prophet = forecast_prophet.merge(
        resampled_df[["ds", "y"]],
        on="ds",
        how="inner",
        suffixes=("_forecast", "_actual"),
    )

    # Calculate evaluation metrics
    total_forecasted_sales = forecast_prophet["yhat"].sum()
    y_true = forecast_prophet["y"].values
    y_pred = forecast_prophet["yhat"].values
    mae = mean_absolute_error(y_true[: len(y_pred)], y_pred)
    mape = (
        (forecast_prophet["yhat"] - forecast_prophet["y"]).abs() / forecast_prophet["y"]
    ).mean()
    mape = np.mean(
        np.abs((y_true - y_pred) / np.where(y_true == 0, 1, y_true))
    )  # Adjusting for zero values
    rmse = np.sqrt(mean_squared_error(y_true[: len(y_pred)], y_pred))

    # Create a table of evaluation metrics
    metrics_table = go.Table(
        header=dict(
            values=["Metric", "Value"], fill_color="paleturquoise", align="left"
        ),
        cells=dict(
            values=[
                ["MAE", "RMSE", "MAPE", "Total Sales", "Total Forecasted Sales"],
                [
                    format_euro(mae),
                    format_euro(rmse),
                    f"{mape:.2%}",
                    format_euro(total_sales),
                    format_euro(total_forecasted_sales),
                ],
            ],
            fill_color="lavender",
            align="left",
        ),
    )

    # Aggregate the forecasted sales by period
    forecast_prophet["month"] = forecast_prophet["ds"].dt.to_period("M")
    forecast_prophet["quarter"] = forecast_prophet["ds"].dt.to_period("Q")
    forecast_prophet["year"] = forecast_prophet["ds"].dt.to_period("Y")

    def create_forecast_table(df: pd.DataFrame, period: str) -> pd.DataFrame:
        forecast_table = (
            df.groupby(period)
            .agg(
                {
                    "y": "sum",
                    "yhat": "sum",
                    "yhat_lower": "sum",
                    "yhat_upper": "sum",
                }
            )
            .reset_index()
        )
        formatted_forecast = forecast_table.copy()

        formatted_forecast["y"] = formatted_forecast["y"].apply(format_euro)
        formatted_forecast["yhat"] = formatted_forecast["yhat"].apply(format_euro)
        formatted_forecast["yhat_lower"] = formatted_forecast["yhat_lower"].apply(
            format_euro
        )
        formatted_forecast["yhat_upper"] = formatted_forecast["yhat_upper"].apply(
            format_euro
        )
        return forecast_table, formatted_forecast

    period_forecast, formatted_period_forecast = create_forecast_table(
        forecast_prophet, sales_grouping_period
    )
    # Create a table for sales_grouping_period forecast
    forecast_table = go.Table(
        header=dict(
            values=[
                sales_grouping_period,
                "Actual Sales",
                "Forecasted Sales",
                "Lower Bound",
                "Upper Bound",
            ],
            fill_color="paleturquoise",
            align="left",
        ),
        cells=dict(
            values=[
                formatted_period_forecast[sales_grouping_period].astype(str),
                formatted_period_forecast["y"],
                formatted_period_forecast["yhat"],
                formatted_period_forecast["yhat_lower"],
                formatted_period_forecast["yhat_upper"],
            ],
            fill_color="lavender",
            align="left",
        ),
    )

    # create plotly figure with subplots
    fig = make_subplots(
        rows=3,
        cols=2,
        subplot_titles=(
            "Total Sales",
            "Filtered Sales",
            "Resampled Sales",
            "Metrics",
            "Sales Forecast",
            f"{sales_grouping_period}ly Forecast",
        ),
        vertical_spacing=0.1,
        horizontal_spacing=0.1,
        specs=[
            [{"type": "scatter"}, {"type": "table"}],
            [{"type": "scatter"}, {"type": "table"}],
            [{"type": "scatter"}, {"type": "table"}],
        ],
        column_widths=[0.75, 0.25],
    )

    # plot resampled sales data on a different plot
    for dim in agg_sales_data[dimension].unique():
        fig.add_trace(
            go.Scatter(
                x=agg_sales_data[agg_sales_data[dimension] == dim][x],
                y=agg_sales_data[agg_sales_data[dimension] == dim][y],
                mode="lines",
                name="Unmodified Sales",
            ),
            row=1,
            col=1,
        )

    fig.add_trace(
        go.Scatter(
            x=resampled_df["ds"],
            y=resampled_df["y"],
            mode="lines",
            name="Resampled Sales",
        ),
        row=2,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecast_prophet["ds"],
            y=forecast_prophet["y"],
            mode="lines",
            name="Actual",
        ),
        row=3,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecast_prophet["ds"],
            y=forecast_prophet["yhat"],
            mode="lines",
            name="Forecast",
        ),
        row=3,
        col=1,
    )

    # show also the uncertainty in the forecast
    fig.add_trace(
        go.Scatter(
            x=forecast_prophet["ds"],
            y=forecast_prophet["yhat_lower"],
            fill=None,
            mode="lines",
            line_color="rgba(0,100,80,0.2)",
            name="Lower Bound",
        ),
        row=3,
        col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=forecast_prophet["ds"],
            y=forecast_prophet["yhat_upper"],
            fill="tonexty",
            mode="lines",
            line_color="rgba(0,100,80,0.2)",
            name="Upper Bound",
        ),
        row=3,
        col=1,
    )

    # Add the tables to the plot
    fig.add_trace(filtered_sales_table, row=1, col=2)
    fig.add_trace(metrics_table, row=2, col=2)
    fig.add_trace(forecast_table, row=3, col=2)

    fig.update_xaxes(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(count=2, label="2y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
    )

    fig.update_layout(
        title=f"Sales Forecast Evaluation - {" - ".join(filters.values())}",
        xaxis_title=x,
        yaxis_title=y,
        showlegend=True,
    )

    fig.show()

    return (
        filtered_sales_data,
        agg_sales_data,
        resampled_df,
        forecast_prophet,
        period_forecast,
        filtered_sales_summary,
    )


#### 1.2.2. Check the sales figures based on start and end date (verify with source)

In [ ]:
sales_data_copy = sales_data.copy()

startDate = '2024-01-01'
endDate = '2024-12-31'
sales_data_filtered = sales_data_copy.query('Date >= @startDate & Date < @endDate').copy()


print(f'{sales_data_filtered['Total Sales'].sum():,.0f}\nfrom: {sales_data_filtered["Date"].min().strftime("%d-%b-%Y")}\nto: {sales_data_filtered["Date"].max().strftime("%d-%b-%Y")}')

# 2. Forecasting

### 2.1. Linear Regression on Historical Sales

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from prophet import Prophet

# Train test split
sales_data_LR_copy = sales_data.copy()
sales_data_LR_copy = sales_data_LR_copy.rename(columns={'Date': 'ds', 'Total Sales': 'y'})

# perform sales forecasting globally without dimensions
global_sales = sales_data_LR_copy.groupby('ds')['y'].sum().reset_index()

train = global_sales[global_sales['ds'] < '2023-06-01']
test = global_sales[global_sales['ds'] >= '2023-06-01']
print(f'Span of Train dataset: {train['ds'].min()} to {train['ds'].max()}')
print(f'Span of Test dataset: {test['ds'].min()} to {test['ds'].max()}')

# fit linear regression model
model_prophet  = Prophet()
model_prophet.fit(train)

df_future = model_prophet.make_future_dataframe(periods=400)

forecast_prophet = model_prophet.predict(df_future)
slim_fc_prophet = forecast_prophet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].round()

# plot sales forecast
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=train['ds'],
    y=train['y'],
    mode='lines',
    name='Train Data'
))
fig.add_trace(
    go.Scatter(
        x=test["ds"],
        y=test["y"],
        mode="lines",
        name="Test Data",
    )
)

fig.add_trace(
    go.Scatter(
        x=forecast_prophet["ds"],
        y=forecast_prophet["yhat"],
        mode="lines",
        name="Forecast",
    )
)

fig.update_layout(
    title='Sales Forecast Evaluation',
    xaxis_title='Date',
    yaxis_title='Total Sales',
)

fig.show()

### 2.2. Ratio Method based on historical sales applied on forecast data

#### 2.2.1. Preprocessing

In [ ]:
historical_sales = sales_data.query('Date >= "2020-01-01" & Date < "2023-07-01"').copy()
forecast = raw_forecast_data.query('Date__c >= "2023-06-01" & Date__c < "2024-07-01"').copy()

In [ ]:
# Extract month and year for grouping
historical_sales['Month'] = historical_sales['Date'].dt.month_name()
forecast['Month'] = forecast['Date__c'].dt.month_name()

#### 2.2.2. Data transformation on sales data

In [ ]:
# Calculate total sales per SKU per grouping
sku_sales_totals = historical_sales.groupby(['Account Id', 'Account Name', 'BL Short', 'Product Family', 'Month', 'Product Id', 'Local Item Code', 'Local Item Description'])['Total Sales'].sum().reset_index()
group_sales_totals = historical_sales.groupby(['Account Id', 'Account Name', 'BL Short', 'Product Family', 'Month'])['Total Sales'].sum().reset_index()

In [ ]:
sku_sales_totals

In [ ]:
# Merge to calculate SKU percentage
sku_sales_percentage = pd.merge(sku_sales_totals, group_sales_totals, on=['Account Id', 'BL Short', 'Product Family', 'Month'], suffixes=('_sku', '_total'))
sku_sales_percentage['SKU_Percentage'] = sku_sales_percentage['Total Sales_sku'] / sku_sales_percentage['Total Sales_total']
sku_sales_percentage

#### 2.2.3. Data transformation on forecast data

In [ ]:
# Merge forecast data with SKU percentage distribution
forecast = forecast.rename(columns={'Account__c': 'Account Id', 'Business_line__c': 'BL Short', 'Product_Family__c': 'Product Family (Account Hierarchy)'})
forecast_sku = pd.merge(forecast, sku_sales_percentage, on=['Account Id', 'BL Short', 'Product Family (Account Hierarchy)', 'Month'], how='left')

In [ ]:
# Allocate forecast amounts to SKUs using the SKU percentage
forecast_sku['Allocated_SKU_Sales'] = forecast_sku['Amount__c'] * forecast_sku['SKU_Percentage']

# Handle cases where there is no historical data
forecast_sku['Allocated_SKU_Sales'] = forecast_sku['Allocated_SKU_Sales'].fillna(0)
forecast_sku.drop(columns=['Account__r.Name',
                           'Account__r.Region__c',
                           'CreatedDate',
                           'CreatedById',
                           'CreatedBy.Name',
                           'Month',], inplace=True)

#### 2.2.4. Forecast Evaluation with actual sales data

In [ ]:
actual_sales = sales_data.query('Date >= "2023-06-01" & Date < "2024-07-01"').copy()
actual_sales['Year-Month'] = actual_sales['Date'].dt.to_period('M')

actual_sku_sales_totals = actual_sales.groupby(['Account Id', 'BL Short', 'Product Family (Account Hierarchy)', 'Year-Month', 'Product Id'])['Total Sales'].sum().reset_index()

forecast_sku['Year-Month'] = forecast_sku['Date__c'].dt.to_period('M')

forecast_sku_actual_sales = pd.merge(forecast_sku, actual_sku_sales_totals, on=['Account Id', 'BL Short', 'Product Family (Account Hierarchy)', 'Year-Month', 'Product Id'], how='left', suffixes=('_forecast', '_actual'))

#### 2.2.5. Methodology Description

1. Get Forecast Data from Salesforce
2. Get Sales Data from Tableau
3. Compute ratio of SKU from sales data for each account - business line - product family - SKU combination based on month of purchase regardless of year to smooth out seasonality
4. Merge with forecast data on account - business line - product family - SKU combination
5. Multiply forecast amount with SKU_Percentage to get forecast amount for each account - business line - product family - SKU combination
6. Get product data from Product object in salesforce and convert euros to material kg based on packaging size.
7. Check how calculated forecast amount compares to actuals

Boundary conditions:
- Dates of sales used for ratio calculation: 2021-01-01 to 2023-06-30
- Dates of forecast used: 2023-07-01 to 2024-06-30
- Dates of actual sales used for evaluation: 2023-07-01 to 2024-06-30

Ideas:
- create streamlit dashboard to have input parameters and show fitting results

### 2.3. Prophet on Historical Sales

#### 2.3.1. Model (in scm_historical_forecast.py)

# 3. Export the results in Excel

In [ ]:
with pd.ExcelWriter(r'C:\Users\Lian.Zhen-Yang\BASF 3D Printing Solutions GmbH\Revenue Operations - General\06. Requests\SNOP FC Breakdown\test_data.xlsx') as writer:
    forecast_sku.to_excel(writer, sheet_name='forecast_sku')
    actual_sku_sales_totals.to_excel(writer, sheet_name='actual_sku_sales')
    forecast_sku_actual_sales.to_excel(writer, sheet_name='forecast_sku_actual_sales')